<a href="https://colab.research.google.com/github/aragaoiberis/Topic_modeling/blob/main/MAT_APL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install wosfile
!pip install flair
!pip install bertopic
!pip install keyphrase-vectorizers
!pip install pickle5
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.1/en_core_sci_sm-0.5.1.tar.gz


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 401 kB 12.8 MB/s 
     |████████████████████████████████| 788 kB 74.9 MB/s 
     |████████████████████████████████| 182 kB 88.4 MB/s 
     |████████████████████████████████| 53 kB 1.9 MB/s 
     |████████████████████████████████| 1.2 MB 75.4 MB/s 
     |████████████████████████████████| 1.6 MB 56.8 MB/s 
     |████████████████████████████████| 5.8 MB 62.5 MB/s 
     |████████████████████████████████| 19.7 MB 1.1 MB/s 
     |████████████████████████████████| 981 kB 51.2 MB/s 
     |████████████████████████████████| 200 kB 67.4 MB/s 
     |████████████████████████████████| 62 kB 1.4 MB/s 
     |████████████████████████████████| 7.6 MB 85.6 MB/s 
  Created wheel for mpld3: filename=mpld3-0.3-py3-none-any.whl size=116702 sha256=051dacc6e20a06edc146e87615c4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 132 kB 12.7 MB/s 
  Created wheel for pickle5: filename=pickle5-0.0.11-cp38-cp38-linux_x86_64.whl size=236290 sha256=b691789403a0205cb5a94fc7eef4d444d47ac34cd0dff41078e9985026591e78
  Stored in directory: /root/.cache/pip/wheels/25/d4/61/dbd8edd1a0d656be7b4267c85db3b61951eb60016a0154a122
Successfully built pickle5
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 15.9 MB 808 kB/s 
  Created wheel for en-core-sci-sm: filename=en_core_sci_sm-0.5.1-py3-none-any.whl size=15870867 sha256=d52210c0239b6ff6b2061ae4aafe52d279bc96479eb64f09e3351f4e706a0e15
  Stored in directory: /root/.cache/pip/wheels/4a/a5/7a/386418a91842b00f876d34427bbe2d8f8d329f626eab850141
Successfully built en-core-sci-sm


In [2]:
!pip install whatlies

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 84 kB 2.8 MB/s 
     |████████████████████████████████| 9.4 MB 33.2 MB/s 
     |████████████████████████████████| 24.2 MB 1.3 MB/s 
     |████████████████████████████████| 295 kB 81.6 MB/s 
     |████████████████████████████████| 965 kB 62.6 MB/s 
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.2.2
    Uninstalling matplotlib-3.2.2:
      Successfully uninstalled matplotlib-3.2.2


In [84]:
!pip install topically[bertopic]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for cohere: filename=cohere-2.9.1-cp38-cp38-linux_x86_64.whl size=9627 sha256=3bd5a1ac06e33d3cbdd8a012031f70fc235190a26a4764284e394f653c1176b5
  Stored in directory: /root/.cache/pip/wheels/c8/79/5b/36adcce533f4b0ed540beef7e6ea61d637a10cb2c8d4670153
Successfully built cohere


In [10]:
!cp  gdrive/MyDrive/CINESHELL/final-model.pt final-model.pt

In [4]:
import torch

In [5]:
import glob
from google.colab import drive
drive.mount('/content/gdrive')
import pandas as pd
import numpy as np
import wosfile

Mounted at /content/gdrive


In [6]:
from whatlies import Embedding, EmbeddingSet

In [11]:
from bertopic import BERTopic
from keyphrase_vectorizers import KeyphraseCountVectorizer

from flair.data import Sentence
from flair.models import SequenceTagger
from flair.tokenization import SciSpacyTokenizer


# load the trained model
model = SequenceTagger.load('final-model.pt')

2022-12-20 23:14:24,185 loading file final-model.pt
2022-12-20 23:14:26,560 SequenceTagger predicts: Dictionary with 35 tags: O, S-MAT, B-MAT, E-MAT, I-MAT, S-PRO, B-PRO, E-PRO, I-PRO, S-DSC, B-DSC, E-DSC, I-DSC, S-CMT, B-CMT, E-CMT, I-CMT, S-SMT, B-SMT, E-SMT, I-SMT, S-APL, B-APL, E-APL, I-APL, S-SPL, B-SPL, E-SPL, I-SPL, S-2, B-2, E-2, I-2, <START>, <STOP>


In [12]:
import pickle5 as pickle
import os

In [13]:
PATH = "/content/gdrive/MyDrive"
file_name = "sent_pChemTok.pickle"
path_to_embed = os.path.join(PATH, file_name)
arq = open(path_to_embed, 'rb')
a = pickle.load(arq)

In [14]:
from tqdm.autonotebook import tqdm

In [85]:
from topically import Topically

In [15]:
for ts in a[:100]:
  for entity in ts.get_spans('ner'):
    if str(entity.get_label("ner").value) == "MAT" or str(entity.get_label("ner").value) == "APL":
      # print entity text, start_position and end_position
      print(f'entity.text is: "{entity.text}"')
      #print(f'entity.start_position is: "{entity.start_position}"')
      #print(f'entity.end_position is: "{entity.end_position}"')

      # also print the value and score of its "ner"-label
      print(f'entity "ner"-label value is: "{entity.get_label("ner").value}"')
      print(f'entity "ner"-label score is: "{entity.get_label("ner").score}"\n')
      print(25*'-')

entity.text is: "Metal - Free A(NH4)X-3"
entity "ner"-label value is: "MAT"
entity "ner"-label score is: "0.4491143859922886"

-------------------------
entity.text is: "Perovskites"
entity "ner"-label value is: "APL"
entity "ner"-label score is: "0.5149492621421814"

-------------------------
entity.text is: "Photovoltaic Applications"
entity "ner"-label value is: "APL"
entity "ner"-label score is: "0.8764469623565674"

-------------------------
entity.text is: "Ferroelectric Photovoltaic Perovskites"
entity "ner"-label value is: "APL"
entity "ner"-label score is: "0.5425700545310974"

-------------------------
entity.text is: "Material Descriptors"
entity "ner"-label value is: "APL"
entity "ner"-label score is: "0.9008031487464905"

-------------------------
entity.text is: "Machine Learning"
entity "ner"-label value is: "APL"
entity "ner"-label score is: "0.6596568524837494"

-------------------------
entity.text is: "Perovskites"
entity "ner"-label value is: "MAT"
entity "ner"-labe

In [16]:
tokens = {} 
tokens_ = {'MAT' : {}, 'SPL' : {}, 'DSC' : {}, 'PRO' : {}, 'APL' : {}, 'SMT' : {}, 'CMT' : {}} 

for sentence in a[:100]:
  for label in sentence.get_spans('ner'):

      # adiciona o embedding de cada token do label 
      # após adicionar, label.get_each_embedding() retorna os tensores
      # e podemos combinar com torch.mean

      for token in label.tokens:
        label.set_embedding(token.text,token.embedding)

      # 0 significa que estamos calculando a média no eixo 0
      # podemos torocar torch.mean por torch.sum
      emb = torch.mean(torch.stack(label.get_each_embedding()),0)

      tokens[label.text] =  Embedding(label.text,emb)

      tokens_[label.tag][label.text] = tokens[label.text]



In [17]:
emb = EmbeddingSet(tokens)

In [18]:
print(tokens)

{'Molecular Design': Emb[Molecular Design], 'Three - Dimensional': Emb[Three - Dimensional], 'Metal - Free A(NH4)X-3': Emb[Metal - Free A(NH4)X-3], 'Perovskites': Emb[Perovskites], 'Photovoltaic Applications': Emb[Photovoltaic Applications], 'Ferroelectric Photovoltaic Perovskites': Emb[Ferroelectric Photovoltaic Perovskites], 'Material Descriptors': Emb[Material Descriptors], 'Machine Learning': Emb[Machine Learning], 'Nontoxic': Emb[Nontoxic], 'Toxic': Emb[Toxic], 'Hybrid': Emb[Hybrid], 'Photovoltaic Devices': Emb[Photovoltaic Devices], 'photovoltaic': Emb[photovoltaic], 'stability properties': Emb[stability properties], 'perovskites': Emb[perovskites], 'convolutional neural networks': Emb[convolutional neural networks], 'photovoltaic perovskites': Emb[photovoltaic perovskites], 'Polar Photovoltaic Effect': Emb[Polar Photovoltaic Effect], 'Chiral Alternating Cations Intercalation': Emb[Chiral Alternating Cations Intercalation], 'Driving Self - Powered Ultraviolet Circularly Polarized

In [19]:
emb2 = EmbeddingSet(tokens_['MAT'])
emb3 = EmbeddingSet(tokens_['APL'])

In [66]:
emb = EmbeddingSet(tokens)
emb.plot_interactive(x_axis=emb["CeTaN3"], y_axis=emb["Hybrid"])

/usr/local/lib/python3.8/dist-packages/whatlies/embedding.py:132: RuntimeWarning: invalid value encountered in float_scalars
  return (self.vector.dot(other.vector)) / (other.vector.dot(other.vector))


alt.LayerChart(...)

In [100]:
topic_model = BERTopic()

In [101]:
tokens = {} 
tokens_ = {'MAT' : {}, 'SPL' : {}, 'DSC' : {}, 'PRO' : {}, 'APL' : {}, 'SMT' : {}, 'CMT' : {}} 

for sentence in a:
  for label in sentence.get_spans('ner'):
        if str(sentence.get_label("ner").value) == "MAT" or str(sentence.get_label("ner").value) == "APL": 
      # adiciona o embedding de cada token do label 
      # após adicionar, label.get_each_embedding() retorna os tensores
      # e podemos combinar com torch.mean

          for token in label.tokens:
        
        
            label.set_embedding(token.text,token.embedding)

      # 0 significa que estamos calculando a média no eixo 0
      # podemos torocar torch.mean por torch.sum
            emb = torch.mean(torch.stack(label.get_each_embedding()),0)

            tokens[label.text] =  Embedding(label.text,emb)

            tokens_[label.tag][label.text] = tokens[label.text]



In [102]:
print(tokens)

{'Ferroelectric Photovoltaic Perovskites': Emb[Ferroelectric Photovoltaic Perovskites], 'Material Descriptors': Emb[Material Descriptors], 'Machine Learning': Emb[Machine Learning], 'photovoltaic perovskites': Emb[photovoltaic perovskites], 'Chalcogenide': Emb[Chalcogenide], 'Perovskites': Emb[Perovskites], 'Perovskite': Emb[Perovskite], 'Chalcohalide': Emb[Chalcohalide], 'Photoabsorbers': Emb[Photoabsorbers], 'Photovoltaic Applications': Emb[Photovoltaic Applications], 'tin': Emb[tin], 'perovskites': Emb[perovskites], 'photovoltaic devices': Emb[photovoltaic devices], 'Organohalide lead': Emb[Organohalide lead], 'photovoltaic applications': Emb[photovoltaic applications], 'Lead - Free Halide': Emb[Lead - Free Halide], 'Double Perovskites': Emb[Double Perovskites], 'Polymer': Emb[Polymer], 'electrolytes': Emb[electrolytes], 'multibandgap hybrid': Emb[multibandgap hybrid], 'Halide Perovskites': Emb[Halide Perovskites], 'Photovoltaic Materials': Emb[Photovoltaic Materials], 'Bonding Mech

In [103]:
embstr = []
for i in tokens:
  embstr.append(str(i))


In [104]:
topics, probs = topic_model.fit_transform(embstr)

In [105]:
app = Topically('PE4bvbL37kORpnVCnTaZBLtfVgbVX4xunKpamWU0')

In [106]:
labels, mapper = app.name_topics((embstr,topics))

In [107]:
topic_model.set_topic_labels(mapper)

In [108]:
topic_model.visualize_barchart(n_words=6,custom_labels=True,width=300,height=300)

In [97]:
topic_model.visualize_documents(embstr,custom_labels=False,hide_annotations=False,width=1200,height=1200)

In [98]:
topic_model.visualize_hierarchy()